In [1]:
import pandas as pd
from logging import root
from PIL import Image
import numpy as np
import sys
import os
import csv
import matplotlib.pyplot as plt
import tensorflow as tf 
from keras import layers 
from keras.callbacks import *
from keras.models import Sequential
from keras.layers import *
from keras.preprocessing.image import ImageDataGenerator
import logging, os

print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
tf.config.list_physical_devices('GPU')

Num GPUs Available:  1


[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [5]:

image_size = (64, 64)
batch_size = 64

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    "/Users/ajones/Documents/asl/ASL_database/ASL_database_train/",
    validation_split=0.2,
    subset="training",
    seed=1337,
    image_size=image_size,
    batch_size=batch_size,
)
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    "/Users/ajones/Documents/asl/ASL_database/ASL_database_train/",
    validation_split=0.2,
    subset="validation",
    seed=1337,
    image_size=image_size,
    batch_size=batch_size,
)

Found 2637 files belonging to 26 classes.
Using 2110 files for training.
Found 2637 files belonging to 26 classes.
Using 527 files for validation.


In [9]:
model=Sequential([])

model.add(Conv2D(64,(3,3),activation="relu",input_shape=(64,64,3)))
model.add(BatchNormalization())
model.add(Conv2D(64,(3,3),activation="relu",input_shape=(64,64,3)))
model.add(BatchNormalization())
model.add(MaxPooling2D(2,2))
model.add(Dropout(.1))

model.add(Conv2D(128,(3,3),activation="relu"))
model.add(BatchNormalization())
model.add(Conv2D(128,(3,3),activation="relu"))
model.add(BatchNormalization())
model.add(MaxPooling2D(2,2))

model.add(Flatten())
model.add(BatchNormalization())
model.add(Dense(256,activation="relu"))
model.add(BatchNormalization())
model.add(Dense(26,activation="softmax"))

model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_8 (Conv2D)           (None, 62, 62, 64)        1792      
                                                                 
 batch_normalization_12 (Bat  (None, 62, 62, 64)       256       
 chNormalization)                                                
                                                                 
 conv2d_9 (Conv2D)           (None, 60, 60, 64)        36928     
                                                                 
 batch_normalization_13 (Bat  (None, 60, 60, 64)       256       
 chNormalization)                                                
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 30, 30, 64)       0         
 2D)                                                             
                                                      

In [ ]:
model.compile(optimizer = 'adam', loss = "sparse_categorical_crossentropy", metrics = ['accuracy'])
hist = model.fit(train_ds, epochs = 20, batch_size = 64, validation_data=val_ds)

Epoch 1/20
33/33 [==============================] - 10s 239ms/step - loss: 1.2989 - accuracy: 0.6536 - val_loss: 4.2507 - val_accuracy: 0.1328
Epoch 2/20
33/33 [==============================] - 9s 236ms/step - loss: 0.4027 - accuracy: 0.9062 - val_loss: 1.8938 - val_accuracy: 0.5256
Epoch 3/20
33/33 [==============================] - 9s 234ms/step - loss: 0.1683 - accuracy: 0.9649 - val_loss: 1.1897 - val_accuracy: 0.6926
Epoch 4/20
33/33 [==============================] - 9s 228ms/step - loss: 0.1063 - accuracy: 0.9754 - val_loss: 0.9274 - val_accuracy: 0.7571
Epoch 5/20
33/33 [==============================] - 9s 226ms/step - loss: 0.0643 - accuracy: 0.9839 - val_loss: 0.6848 - val_accuracy: 0.8368
Epoch 6/20
33/33 [==============================] - 9s 227ms/step - loss: 0.0414 - accuracy: 0.9924 - val_loss: 0.8824 - val_accuracy: 0.7761
Epoch 7/20
33/33 [==============================] - 9s 227ms/step - loss: 0.0181 - accuracy: 0.9962 - val_loss: 0.7495 - val_accuracy: 0.8102
Epoch

In [ ]:
score = model.evaluate(x = X_test, y = y_test, verbose = 0)
print('Accuracy for test images:', round(score[1]*100, 3), '%')
score = model.evaluate(x = X_eval, y = y_eval, verbose = 0)
print('Accuracy for evaluation images:', round(score[1]*100, 3), '%')

In [ ]:
saved_model_path = "./asl_model"
model.save(saved_model_path)
logging.disable(logging.WARNING)
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"
command = f"python -m tf2onnx.convert --saved-model ./asl_model --opset 18 --output asl.onnx"
os.system(command)